## remove docker 

In [ ]:
%%bash

docker rm -f $(docker ps -a -q)
docker rmi -f $(docker images -a -q)
echo "done"
#!docker system prune

### Get the Greengrass software on the RPi

Download and unzip the Greengrass core software.

In [ ]:
!echo "downloading" && curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip -o greengrass-nucleus-latest.zip -d ~/GreengrassCore && echo "done"

### Register the RPi with AWS using Greengrass

You need to paste in your AWS credentials from the workshop authentication page.

Run the Greengrass provisioning command.

In [ ]:
%%bash
# add your credential from AWS console
export AWS_ACCESS_KEY_ID=""
export AWS_SECRET_ACCESS_KEY=""
export AWS_SESSION_TOKEN=""
export AWS_DEFAULT_REGION="us-east-1"

sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ../GreengrassCore/lib/Greengrass.jar \
           --thing-name ROS2_Minipupper \
           --thing-group-name ROS2_Minipuppers \
           --tes-role-name LabRole \
           --tes-role-alias-name GreengrassCoreTokenExchangeRoleAlias \
           --component-default-user ggc_user:ggc_group \
           --provision true \
           --setup-system-service true \
           --deploy-dev-tools true \
           --aws-region ${AWS_DEFAULT_REGION}

echo "done"

### Empower ggc_user to make full use of the Pupper's systems

Give the newly created ggc_user access to run docker applications and the i2c bus:

In [ ]:
%%bash

sudo usermod -aG docker ggc_user
sudo usermod -aG i2c ggc_user
sudo usermod -aG video ggc_user
sudo setfacl --modify group:docker:rw /var/run/docker.sock
echo "done"

### Check the RPi has appeared in the AWS Console

In a new browser tab, open the Greengrass Console and click on Core Devices. You should see your new Greengrass application running!

### Use the Greengrass CLI to see what the Pupper is capable of

Note: The CLI deployment takes ~4 minutes, if you see errors here, wait a few minutes and try again.

Run the following command to interact with the Greengrass CLI on the robot:

In [ ]:
%%bash

cd /greengrass/v2/bin
sudo ./greengrass-cli component list

## check the output from docker-compose process

In [ ]:
%%bash
export COMPONENT_NAME=com.example.ros2.mini_pupper_v2
#export COMPONENT_NAME=com.example.ros2.demo

export ARTIFACT_DIR=/greengrass/v2/packages/artifacts/$COMPONENT_NAME/1.0.6/
sudo docker-compose -f $ARTIFACT_DIR/docker-compose_nobuild.yaml logs --follow

### Done!

You can now return to the workshop documentation.

In [ ]:
%%bash
export COMPONENT_NAME=com.example.ros2.mini_pupper_v2
#export COMPONENT_NAME=com.example.ros2.demo

export ARTIFACT_DIR=/greengrass/v2/packages/artifacts/$COMPONENT_NAME/1.0.2/

sudo docker-compose -f $ARTIFACT_DIR/docker-compose_nobuild.yaml down